# Cogniac Python SDK #

In [1]:
from cogniac import CogniacConnection

In [2]:
tenant_id = 'jbq93i0pqabb'  # Getting Started tenant ID

In [3]:
cc = CogniacConnection(tenant_id=tenant_id) # COG_USER and COG_PASS are set

In [4]:
tenant = cc.get_tenant()
applications = cc.get_all_applications()
subjects = cc.get_all_subjects()

In [5]:
tenant

Cogniac - Getting Started (jbq93i0pqabb)

In [6]:
applications

[amytest (edehs8zr),
 Detect Pliers (nrpgjopr),
 Detect Pliers (s4n8palr),
 Detect Pliers (z2q7ijqs),
 Detect4 (feiizwbq),
 Detection2 (tgd4facd),
 detection3 (jgtvg1dt)]

In [7]:
subjects

[Plier (plier_3bcdswen),
 Plier (plier_5pkwtwld),
 Plier And Needle Plier (plier_and_needle_plier_1mj),
 Plier2 (plier2_6eczjec),
 test (test_6zgejojdv),
 test-sub (testsub_8moowt),
 test=sub2 (testsub2_6mkcm),
 Tool Box (tool_box_9uheh)]

In [8]:
active_applications = [app for app in applications if app.active]
active_applications

[amytest (edehs8zr),
 Detect Pliers (z2q7ijqs),
 Detect4 (feiizwbq),
 Detection2 (tgd4facd),
 detection3 (jgtvg1dt)]

In [9]:
application_id = 'z2q7ijqs'

In [10]:
application = cc.get_application(application_id)
application

Detect Pliers (z2q7ijqs)

In [11]:
subject_id = application.output_subjects[0]
subject_id

'plier_3bcdswen'

In [12]:
subject = cc.get_subject(subject_id)
subject

Plier (plier_3bcdswen)

In [13]:
subject.media_associations()

<generator object CogniacSubject.media_associations at 0x000001FD9B7229E0>

In [14]:
all_media_associations = [association for association in subject.media_associations()]

In [15]:
media_association = all_media_associations[0]
media_association

{'other_media': [],
 'media': {'media_id': 'XF2UTKUPEK3OWNOIA9BQSYLEF07T'},
 'subject': {'media_id': 'XF2UTKUPEK3OWNOIA9BQSYLEF07T',
  'probability': 0.000995069883951259,
  'focus': None,
  'app_data_type': None,
  'updated_at': 1622821928.748013,
  'consensus': None,
  'app_data': None,
  'subject_uid': 'plier_3bcdswen'},
 'focus': None,
 'media_list': [{'media_id': 'XF2UTKUPEK3OWNOIA9BQSYLEF07T'}]}

In [16]:
media_id = media_association['media']['media_id']
media_id

'XF2UTKUPEK3OWNOIA9BQSYLEF07T'

In [17]:
media = cc.get_media(media_id)
media

In [18]:
media.filename

'Image__2019-05-24__14-37-40.png'


# Cogniac REST API #

In [19]:
import os
import requests

In [20]:
COG_URL_PREFIX = os.environ.get('COG_URL_PREFIX')
COG_URL_PREFIX

'https://api.cogniac.io/1'

In [21]:
COG_USER = os.environ.get('COG_USER')
COG_PASS = os.environ.get('COG_PASS')
COG_API_KEY = os.environ.get('COG_API_KEY')

In [22]:
resp = requests.get(f'{COG_URL_PREFIX}/version')
resp.json()

{'build': '1e7d3ba', 'api_version': '1'}

In [23]:
auth = (COG_USER, COG_PASS)  # authorization tuple when using username and password

In [24]:
tenants = requests.get(f'{COG_URL_PREFIX}/users/current/tenants', auth=auth).json()['tenants']
type(tenants)

list

In [25]:
tenants[0].keys()

dict_keys(['tenant_id', 'name', 'roles'])

### Retrieve OAuth Token for a specific tenant ###

In [26]:
authorization = requests.get(f'{COG_URL_PREFIX}/token?tenant_id={tenant_id}', auth=auth).json()
auth_header = {'Authorization' : f'Bearer {authorization["access_token"]}'}

### Use Token to Retrieve Authenticated Tenant ###

In [27]:
auth_tenant = requests.get(f'{COG_URL_PREFIX}/tenants/current', headers=auth_header).json()
auth_tenant

{'edgeflow_rancher_endpoint': None,
 'media_storage_config': None,
 'default_inference_execution_policies': None,
 'created_by': 'amy@cogniac.co',
 'azure_sas_tokens': None,
 'description': None,
 'contract_max_deployment_groups': None,
 'beta': None,
 'edgeflow_monitoring_sla': None,
 'name': 'Cogniac - Getting Started',
 'edgeflow_rancher_user_token': None,
 'dedicated_training_instances': None,
 'tenant_id': 'jbq93i0pqabb',
 'created_at': 1620313987.052882,
 'modified_at': 1620313987.052882,
 'contract_end_date': None,
 'custom_data': None,
 'notification_config': None,
 'default_subject_expires_in_months': None,
 'media_storage_limit_GB': 1000,
 'region': None,
 'has_ops': None}

### Example: Get application events ###

In [28]:
events_url = f'{COG_URL_PREFIX}/applications/{application_id}/events'
events_url

'https://api.cogniac.io/1/applications/z2q7ijqs/events'

In [29]:
# Utility function to retrieve all data if pagination is required

def get_all_data(url):
    all_data = []
    while url:
        resp = requests.get(url, headers=auth_header).json()
        all_data.extend(resp['data'])
        url = resp['paging'].get('next')
    return all_data

In [30]:
events = get_all_data(events_url)
events

[{'application_id': 'z2q7ijqs',
  'event_id': 'XQ22THoY',
  'created_at': 1622216769.256048,
  'message': 'model_id InitialModel_z2q7ijqs_lpnVLd.tgz, validation_media_focus_size 0',
  'type': 'model_release',
  'event_data': '{"model_uri": "s3://cogniac-models/InitialModel_z2q7ijqs_lpnVLd.tgz", "model_id": "InitialModel_z2q7ijqs_lpnVLd.tgz", "timestamp": 1622216769.055984, "validation_data_size": 0, "app_id": "z2q7ijqs", "validation_media_focus_size": 0, "output_subjects": ["plier_3bcdswen"], "tenant_id": "jbq93i0pqabb", "type": "box_detection", "release_metrics": "best_F1"}'},
 {'application_id': 'z2q7ijqs',
  'event_id': 'vUmJhdYt',
  'created_at': 1622821911.370801,
  'message': 'kapil.kaisare@cogniac.co requested replay of 20 subject-media from subject tool_box_9uheh in application z2q7ijqs, replay order random, probability lower 0.50, upper 1.00, time range from Jan 01, 1970 12:00:00 AM to Jun 04, 2021 03:51:51 PM UTC, force_feedback False, validation_only False',
  'type': 'appli